# Demo Lakehouse

El presente demo muestra a nivel conceptual como funciona una arquitectura Bronze-Silver-Gold.

En el demo, las capas están divididas de la siguiente forma:

* **Bronze**  : ingesta cruda, en este caso es de archivos CSV - normalmente vienen de un sistema externo
* **Silver**  : transformaciones de datos
* **Gold**    : clasificación + agregaciones pensando en las vistas que utilizarían los usuarios finales.

Dataset a utilizar: *winequality-red.csv*

In [0]:
# El dataset de Wine Quality está disponible en Databricks dentro de los datasets de ejemplo
file = dbutils.fs.ls("/databricks-datasets/wine-quality/winequality-red.csv")

# Capa Silver

Limpieza de datos, normalización, y transformaciones

Transformaciones:
* Columna constante wine_type = 'red'
* total_acidity = fixed_acidity + volatile_acidity
* Limpieza de nulos

In [0]:
# Abrimos la tabla creada en el notebook anterior "Bronze"
df_silver = spark.table("big_data_ii_2025.spark_examples.wine_bronze")
df_silver.printSchema()

Transformaciones:

* Agregar una columna `wine_type` a cada fila. Con un valor constante de `red`

* Agregar una columna `total_acidity` que consista en la suma de las columnas `fixed_acidity` + `volatile_acidity`

* Limpieza de nulos en la columna `quality`

In [0]:
from pyspark.sql.functions import col, lit
df_silver = (df_silver
    .withColumn("wine_type", lit("red"))
    .withColumn("total_acidity",
                col("fixed_acidity") + col("volatile_acidity"))
    .filter(col("quality").isNotNull()))
df_silver.display()

Y lo salvamos como una tabla en el catalogo

In [0]:
(df_silver
 .write
 .format("delta")
 .mode("overwrite")
 .saveAsTable(f"big_data_ii_2025.spark_examples.wine_silver"))